# Refactor Public Goods Game


## Overall TODOS

In [ ]:
#TODO Implement improved player-types (altruist and freerider) for split endowment case (make them a little more responsive versus totally static and identical) 
# aka varying degrees of altruism : debating this one – for behavioral / lab results reasons 

#TODO
# I now want to change the type of randomness used. Rather than a uniform distribution, maybe something with a tail, etc. 

#TODO Behavior Issue. Check current output. Currently, very drastic changes in the first round, but then almost no changes occur later on. This should be worked on.

#TODO Additional issue : I do not think my scale factor is not working properly

#TODO Once I do the above, I can make better graphs that show (1) average group level cooperation, (2) average player level coop, (3) average round level coop, etc.

#TODO design in a way to allow for specific comparissons between the nature of the endowment (SHARED == 0/1) with same parameter sets 

#TODO parameterize to explore the parameter space

#TODO implement power analysis by estimating effect size given the information that I am storing 

#TODO I would be interested in creating some visual component for displaying the change of cooperation of time given different parameter sets -- potentially with
# a scroller feature 

# Import Utils

In [1]:
from src.plots import *

In [ ]:
groups_example = formed_groups_matrix(GROUP_SIZE, NUM_GROUPS)
np.argwhere(groups_example == 1) # GOOD FOR VECTORIZING LATER; gives me indices of what groups ppl are in. Will no longer need to use for loop

In [ ]:
test_mc = monte_carlo_simulation(SIMS)
print(test_mc)

## Currently:
- Working on Functions to make meaningful graphs from the data stored in 'metrics_of_interest.csv' 

In [12]:
metrics_of_interest = pd.read_csv('metrics_of_interest.csv')

### What visuals do I want to make with the metric_of_interest data?
- expected contribution from each player (0-15) in each round 
    - also average palyer contribution across rounds would be interesting 
- expected contribution of each group (0-7) in each round 
    - also average group contribution across rounds would be interesting

In [13]:
metrics_of_interest.head(35)

,Unnamed: 0,SimulationID,RoundID,group0,group1,group2,group3,group4,group5,group6,group7,playeravg
0,player0,0.0,0.0,43.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,21.500
1,player1,0.0,0.0,48.0,NaN,1.00,NaN,NaN,NaN,NaN,NaN,24.500
2,player2,0.0,0.0,33.0,NaN,NaN,5.00,NaN,NaN,NaN,NaN,19.000
3,player3,0.0,0.0,14.0,NaN,NaN,NaN,1.00,NaN,NaN,NaN,7.500
4,player4,0.0,0.0,NaN,26.0,NaN,NaN,NaN,5.00,NaN,NaN,15.500
5,player5,0.0,0.0,NaN,25.0,NaN,NaN,NaN,NaN,17.0,NaN,21.000
6,player6,0.0,0.0,NaN,23.0,NaN,NaN,NaN,NaN,NaN,17.00,20.000
7,player7,0.0,0.0,NaN,NaN,4.00,NaN,NaN,10.00,NaN,NaN,7.000
8,player8,0.0,0.0,NaN,NaN,7.00,NaN,NaN,NaN,33.0,NaN,20.000
9,player9,0.0,0.0,NaN,NaN,46.00,NaN,NaN,NaN,NaN,2.00,24.000
